In [1]:
!pip3 install stellargraph
!pip3 install node2vec

     |████████████████████████████████| 435 kB 8.0 MB/s 
     |████████████████████████████████| 462 kB 49.7 MB/s 
  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=73116bfb66e1ea023940a3ac8c3afb21ce6c2554f67397f1a063dbbae55d9815
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec


In [2]:
# split the data and create training and test reduced subgraphs
from stellargraph.data import EdgeSplitter
from stellargraph import StellarGraph
import networkx as nx
from sklearn.model_selection import train_test_split

In [6]:
# G = nx.read_edgelist("facebook_combined.txt", create_using=nx.Graph(), nodetype=int)
G = nx.read_edgelist("107.edges", create_using=nx.Graph(), nodetype=int)
# create splitter instance for subgraphs
edgeSplitter = EdgeSplitter(G)
# split data for test
graph_test, samples_test, labels_test = edgeSplitter.train_test_split(p=0.1, method="global", seed=24)
# g and g_master
edgeSplitter = EdgeSplitter(graph_test, G)
# split data for train
graph_train, samples_train, labels_train = edgeSplitter.train_test_split(p=0.1, method="global", seed=24)


FileNotFoundError: ignored

In [ ]:
# explore the subgraphs a bit
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
nx.draw_networkx(graph_train)
plt.show()

### Node2Vec without supervision

In [ ]:
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
# generate node embedding for graph
node_vec = Node2Vec(graph_train)
node_model = node_vec.fit()
# generating an embedding for each pair of embedded nodes
edges_embs = HadamardEmbedder(keyed_vectors=node_model.wv)
# traverse to get all the node embeddings
train_embs = [edges_embs[(str(x[0]), str(x[1]))] for x in graph_train]
test_embs = [edges_embs[(str(x[0]), str(x[1]))] for x in graph_test]


# import the class
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rf = RandomForestClassifier(n_estimators=10)
rf.fit(train_embs, labels_train)

# predict
y_pred = rf.predict(test_embs)
print('Precision:', metrics.precision_score(labels_test, y_pred)) 
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred))


### GraphSAGE


In [ ]:
# the network requires features, have to make fake features instead
import numpy as np

eye_array = np.eye(graph_train.number_of_nodes())
fake_train_features = [ for node in G.]

### Hand-crafted features